In [82]:
import os as os
import tkinter as tk
from tkinter import filedialog
import string
from PIL import Image
import datetime
import shutil

### IMPORT FILES AND CREATE FILENAMES/LIST 
#select directory with image files
scriptdir = os.path.dirname(os.getcwd()) # get the script directory
root = tk.Tk()
root.withdraw()
filedir = tk.filedialog.askdirectory(initialdir  = scriptdir) # manually select the directory with the files to rename
#os.chdir(filedir) # set the directory with files as current directory
motherdir = os.path.dirname(filedir)
parts = filedir.split(os.sep)
filenameroot = parts[-2]
positions = [pos for pos, char in enumerate(filenameroot) if char == '_']
filenameroot_short = filenameroot[positions[0]+1:positions[4]] + '_' + filenameroot[positions[-4]+1:positions[-3]]
starttime = filenameroot[positions[-1]+1:]
startdate = filenameroot[positions[-2]+1:positions[-1]]


def create_datetime(date_str, time_str):
    # Extract date components
    year = int(date_str[0:4])
    month = int(date_str[4:6])
    day = int(date_str[6:8])
    
    # Extract time components
    hour = int(time_str[0:2])
    minute = int(time_str[2:4])
    second = int(time_str[4:6])
    
    # Create datetime object
    return datetime.datetime(year, month, day, hour, minute, second)

startdt = create_datetime(startdate, starttime)
print('start time:')
print(startdt)

# list all tiff files in the directory
filelist = [f for f in os.listdir(filedir) if f.endswith('.tiff') and '._' not in f] # make a list of all tsv files in data directory
#filelist = sorted(filelist) #sort files by name

# sort files by time
filelist = [os.path.join(filedir, f) for f in filelist] #add path to file name
filelist.sort(key=os.path.getmtime) #sort files by time
filelist = [os.path.basename(f) for f in filelist] #remove path from filename
numfiles = len(filelist)
print('number of files:')
print(numfiles)


start time:
2024-04-29 16:39:47
number of files:
1


In [83]:
### EXTRACT THE CREATION TIME OF THE FILES VIA TIME MODIFIED
def get_creation_times(filedir, filelist):
    creation_times = {}
    for filename in filelist:
        # Construct the full path to the file
        file_path = os.path.join(filedir, filename)
        
        # Get the file stats
        try:
            stat_info = os.stat(file_path)
            # Extract the creation time and convert to a readable format
            creation_time = datetime.datetime.fromtimestamp(stat_info.st_mtime)
            creation_times[filename] = creation_time
        except AttributeError:
            # Handle the case where st_birthtime is not available
            creation_times[filename] = "Creation time not supported on this platform"
        except FileNotFoundError:
            # Handle the case where the file does not exist
            creation_times[filename] = "File not found"

    return creation_times

creation_times = get_creation_times(filedir, filelist)
#for file, time in creation_times.items():
    #print(f"{file}: {time}")

first_datetime = creation_times['1.tiff']
print('first file time: ')
print(first_datetime)

timeperplate = first_datetime - startdt;
print('time per plate: ')
print(timeperplate)



first file time: 
2024-04-29 16:57:28
time per plate: 
0:17:41


In [84]:
#subtract half of the time of acquisition per plate
# Iterate through the dictionary and update each datetime
updated_file_times = {key: value - timeperplate/2 for key, value in creation_times.items()}

# Print the updated dictionary to verify
for file, new_time in updated_file_times.items():
    print(f"{file}: {new_time}")

1.tiff: 2024-04-29 16:48:37.500000


In [85]:
#CREATE NEW FILENAMES CONTAINING THE TIME OF ACQUISITION  
midphrase = [filelist[j][0:-5] for j in range(0,len(filelist))]
for j in range(0,len(filelist)):
    if len(filelist[j][0:-5]) == 1:
        midphrase[j] = '0'+filelist[j][0:-5]
filename_new = [updated_file_times[filelist[j]].strftime("%Y%m%d_%H%M%S")+ '_' + filenameroot_short + '_' + midphrase[j] +  '.tiff' for j in range(0,len(filelist))]
print(filename_new)

['20240429_164837_GV41_d6_PL1_snap_FL5p_01.tiff']


In [86]:
#### Create a copy of teh tiffs with new name including time in collected folder

def copy_tiff_file(source_path, destination_folder, filenew, fileold):
    # Extract filename from the source path
    filename = filenew
    
    # Create the destination path
    destination_path = os.path.join(destination_folder, filenew)
    
    # Check if the source file exists
    if not os.path.isfile(source_path):
        return "Source file does not exist."

    # Check if the destination directory exists, if not, create it
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
        print(f"Destination directory '{destination_folder}' created.")

    # Copy the file to the new location
    shutil.copy(source_path, destination_path)
    return f"File copied to {destination_path}"

destination_dir = os.path.join('/'.join(parts[0:-2]), 'GV41_PL1_d04-d06_collected_FL5_snaps')

for i in range (0, len(filename_new)):
    source_tiff_path = os.path.join(filedir,filelist[i])
    result = copy_tiff_file(source_tiff_path, destination_dir, filename_new[i],  filelist[i])
    print(result)


File copied to /Volumes/MUrb_PDN/AutoLCI/01_Gastruloids/20240425-29_GV41_d02-d06_EzrCA_DoxWithd/GV41_PL1_d04-d06_collected_FL5_snaps/20240429_164837_GV41_d6_PL1_snap_FL5p_01.tiff
